# 构建一个聊天机器人

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

In [16]:
chat = ChatOpenAI(
    model="deepseek-chat",
    temperature=0.3,
    max_tokens=200,
    api_key='sk-6a82d1572f264b75952c7db2c081ef4c', # Deepseek api key
    base_url="https://api.deepseek.com/v1"
)

messages = [
    HumanMessage(content="Hi! I'm Bob"),
]

response = chat.invoke(messages)

print(response.content)

Hi Bob! 👋 Nice to meet you! How can I help you today? 😊


模型本身没有任何状态概念。例如，如果您问一个后续问题：

In [20]:
messages = [
    HumanMessage(content="What's my name?"),
]
response = chat.invoke(messages)

print(response.content)

I don’t have access to personal information about you unless you share it with me. If you’d like, you can tell me your name, and I’ll happily use it in our conversation! 😊


我们可以看到它没有将之前的对话轮次作为上下文，因此无法回答问题。

若加上之前的聊天记录：

In [27]:
messages = [
    HumanMessage(content="What's my name?"),
    SystemMessage(content="Hello Bob! How can I assist you today?"),
    HumanMessage(content="What's my name?"),
]
response = chat.invoke(messages)

print(response.content)

Your name is Bob! You mentioned it at the beginning of our conversation: "Hello Bob!"  

Is there anything else you'd like help with? 😊


现在我们可以看到我们得到了一个好的回应！

这是支撑聊天机器人进行对话交互的基本理念。 那么我们如何最好地实现这一点呢？

### 消息历史

我们可以使用消息历史类来包装我们的模型，使其具有状态。 这将跟踪模型的输入和输出，并将其存储在某个数据存储中。 未来的交互将加载这些消息，并将其作为输入的一部分传递给链。

1. 确保安装 langchain-community，因为将使用其中的集成来存储消息历史。(!pip install langchain_community)

2. 之后，导入相关类并设置我们的链，该链包装模型并添加当前对话的消息历史。get_session_history作为传入的函数，预计接受一个session_id并返回一个消息历史对象。session_id用于区分不同对话，并作为配置的一部分在调用新链时传入。（一个对话一个链一个session_id）

In [42]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(chat, get_session_history) # chat是之前调用的大模型

3. 创建一个config，用于配置一个session_id，让大模型知道要读取哪一段消息历史。

In [46]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)

response.content

'Hi Bob! 👋 Nice to meet you. How can I help you today? 😊'

In [48]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is **Bob**—unless you’ve changed it in the last few seconds! 😄 What’s up, Bob? Anything fun or interesting on your mind?'

如果我们更改配置以引用不同的 session_id，我们可以看到它开始新的对话。

In [51]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'I don’t have access to personal information about you unless you share it with me. If you’d like, you can tell me your name, and I’ll be happy to use it in our conversation! 😊'

而且，我们始终可以回到原始对话（因为我们将其保存在数据库中）

In [54]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"Still **Bob**! Unless you're secretly a undercover agent with a codename—then I might need a hint. 😉  \n\nWhat’s the plan today, Bob? Or should I call you *Agent B* now? 🕵️\u200d♂️"

现在，我们所做的只是为模型添加了一个简单的持久化层（将聊天机器人和用户的对话历史进行存储并保持的机制，使对话不会丢失）。我们可以通过添加提示词模板来使其变得更加复杂和个性化。

### 提示词模板

提示词模板帮助将原始用户信息转换为大型语言模型可以处理的格式。在这种情况下，原始用户输入只是一个消息，我们将其传递给大型语言模型。现在让我们使其变得更复杂一些。首先，让我们添加一个带有一些自定义指令的系统消息（但仍然将消息作为输入）。接下来，我们将添加除了消息之外的更多输入。

1. 首先，让我们添加一个系统消息。为此，我们将创建一个 ChatPromptTemplate。我们将利用 MessagesPlaceholder 来传递所有消息。

*chain = prompt | model*：通过 “|” 运算符将提示模板prompt与语言模型model串联成一个链chain。这意味着在调用chain时，会先将输入数据传入prompt进行处理（生成包含系统消息和具体对话消息的完整提示），再将处理结果传递给model进行推理，最终得到 AI 的回复。

In [68]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"), # 表示后续会通过键为 “messages” 的参数传入具体消息
    ]
)

chain = prompt | chat

2. 传递一个包含 messages 键的字典，其中包含一系列消息，而不是传递消息列表。

In [70]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})

response.content

"Hi Bob! 👋 How's it going? What can I help you with today? 😊"

3. 现在，将其包装在消息历史对象中，与大模型对话

In [77]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)
config = {"configurable" : {"session_id" : "abc5"}}

response = with_message_history.invoke(
    [HumanMessage(content = "I'm Jim")],
    config = config
)

response.content

"Hi Jim! How can I assist you today? Whether you have questions, need help with something, or just want to chat, I'm here for you. Let me know what's on your mind!"

In [79]:
response = with_message_history.invoke(
    [HumanMessage(content = "What's my name?")],
    config = config
)

response.content

'Your name is **Jim**! You mentioned it right at the beginning: *"I\'m Jim."*  \n\nLet me know if there\'s anything else you\'d like help with, Jim! 😊'

✳现在将系统提示改变的更复杂一点

1. 改变提示词，并在提示词中加入{language}变量，使得我们可以自定义变量值

In [140]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | chat

2. 包装在消息历史对象中。因为多了一个输入的键，需要明确指定哪一个键来保存聊天历史

In [142]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages",)
config = {"configurable": {"session_id": "abc10"}}

response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="hi! I'm todd")], 
        "language": "Spanish"
    },
    config=config,
)

response.content

'¡Hola, Todd! ¿Cómo estás? 😊 ¿En qué puedo ayudarte hoy?  \n\n*(Translation: Hi, Todd! How are you? 😊 How can I help you today?)*  \n\nFeel free to ask me anything in Spanish or English—I’m happy to help!'

### 管理对话历史

构建聊天机器人时，一个重要的概念是如何管理对话历史。如果不加以管理，消息列表将无限增长，并可能溢出大型语言模型的上下文窗口。因此，添加一个限制您传入消息大小的步骤是很重要的。

❗重要的是，此管理对话历史的步骤，需要在提示模板之前，以及获取所有对话历史之后，再执行

因为deepseek中没有token计数的方法，所以我们先自定义一个方法，如下：

In [150]:
!pip install tiktoken

In [152]:
from typing import List

# pip install tiktoken
import tiktoken
from langchain_core.messages import BaseMessage, ToolMessage


def str_token_counter(text: str) -> int:
    enc = tiktoken.get_encoding("o200k_base")
    return len(enc.encode(text))


def tiktoken_counter(messages: List[BaseMessage]) -> int:
    """Approximately reproduce https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb

    For simplicity only supports str Message.contents.
    """
    num_tokens = 3  # every reply is primed with <|start|>assistant<|message|>
    tokens_per_message = 3
    tokens_per_name = 1
    for msg in messages:
        if isinstance(msg, HumanMessage):
            role = "user"
        elif isinstance(msg, AIMessage):
            role = "assistant"
        elif isinstance(msg, ToolMessage):
            role = "tool"
        elif isinstance(msg, SystemMessage):
            role = "system"
        else:
            raise ValueError(f"Unsupported messages type {msg.__class__}")
        num_tokens += (
            tokens_per_message
            + str_token_counter(role)
            + str_token_counter(msg.content)
        )
        if msg.name:
            num_tokens += tokens_per_name + str_token_counter(msg.name)
    return num_tokens

LangChain 提供了一些内置的助手来 管理消息列表。在这种情况下，我们将使用 trim_messages 助手来减少我们发送给模型的消息数量。修剪器允许我们指定希望保留的令牌数量，以及其他参数，例如是否希望始终保留系统消息以及是否允许部分消息：

In [168]:
from langchain_core.messages import AIMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,       # 规定了修剪后消息的最大令牌数
    strategy="last",     # 表示保留最近的消息
    token_counter=tiktoken_counter,  # 指定用所使用的语言模型来计算令牌数
    include_system=True, # 确保系统消息始终被保留
    allow_partial=False, # 表示不允许保留不完整的消息
    start_on="human",    # 指定从人类消息开始处理
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

现在如果我们尝试询问模型我们的名字，它将不知道，因为我们修剪了聊天历史的那部分:

In [170]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer) # 作用是在链中添加一个处理步骤，对输入的消息列表进行修剪
    | prompt
    | chat
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

'I don’t have access to personal information, so I don’t know your name. But I’d be happy to call you whatever you like—just let me know! 😊'

In [176]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what question did I ask?")],
        "language": "English",
    }
)
response.content

'You asked, *"whats 2 + 2"*, and I answered *"4"*. Then you said *"thanks"*, and I replied *"no problem!"*. After that, you asked *"having fun?"*, and I said *"yes!"*.  \n\nNow you’re asking *"what question did I ask?"* — which is this very meta moment! 😄  \n\nLet me know what’s next!'

现在，我们将修剪器包装在消息历史中：

In [180]:
chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer) # 作用是在链中添加一个处理步骤，对输入的消息列表进行修剪
    | prompt
    | chat
)

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

In [182]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="what question did I ask?")],
        "language": "English",
    },
    config=config,
)

response.content

'You asked, *"whats 2 + 2"*, and I answered *"4"*. Then you said *"thanks"*, and I replied *"no problem!"*. After that, you asked *"having fun?"*, and I said *"yes!"*.  \n\nNow you just asked: *"what question did I ask?"*  \n\nLet me know if you\'d like to dive deeper into any of these! 😊'

### 流式处理

大型语言模型有时可能需要一段时间才能响应，因此为了改善用户体验，大多数应用程序所做的一件事是随着每个令牌的生成流回。这样用户就可以看到进度。

所有链都暴露一个.stream方法，使用消息历史的链也不例外。我们可以简单地使用该方法获取流式响应。

In [187]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
        "language": "English",
    },
    config=config,
):
    print(r.content, end="|")

|Hi| Todd|!| Here|'s| a| joke| for| you|:|  

|**|Why| don|’|t| skeletons| fight| each| other|?|**|  
|*|Because| they| don|’|t| have| the| guts|!|*|  

|Hope| that| gives| you| a| chuckle|!| 😄| Got| any| favorite| joke| topics|?||

### 总结

* 想要对话带历史对话，封装入消息历史对象中，此时需要配置session_id，invoke函数中要传入config，指定调用哪一个对话历史。